In [1]:
import os 
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms,models

In [3]:
class ChestDataset(Dataset):
    def __init__(self, data_dir, classes, transform=None):
        self.data_dir = data_dir
        self.classes = classes
        self.transform = transform

        # collect image paths and labels
        self.image_paths = []
        self.labels = []
        for label, class_name in enumerate(classes):
            class_dir = os.path.join(data_dir, class_name)
            for file in os.listdir(class_dir):
                if file.lower().endswith((".jpg", ".png", ".jpeg")):
                    self.image_paths.append(os.path.join(class_dir, file))
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]

        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label


In [4]:
classes=["adenocarcinoma","large.cell.carcinoma","normal","squamous.cell.carcinoma"]

data_dir="./Chest-CT-Scan/Data"
train_dir=os.path.join(data_dir,"train")
test_dir=os.path.join(data_dir,"test")
val_dir=os.path.join(data_dir,"valid")



In [5]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std=[0.229,0.224,0.225]),
])
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std=[0.229,0.224,0.225]),
])

train_ds = ChestDataset(train_dir, classes, transform=train_transform)
val_ds = ChestDataset(val_dir, classes, transform=val_transform)
test_ds = ChestDataset(test_dir, classes, transform=val_transform)

num_workers = 0  # safer for Windows
batch_size = 32

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [6]:
model=models.resnet18(pretrained=True)
num_ftrs=model.fc.in_features
model.fc=nn.Linear(num_ftrs,len(classes))
model=model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=1e-4)
scheduler=optim.lr_scheduler.ReduceLROnPlateau(optimizer,"min",patience=3)


c:\Users\vedan\.conda\envs\dl-gpu\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\vedan\.conda\envs\dl-gpu\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
best_val_loss=float("inf")
num_epochs=10

In [8]:
from tqdm import tqdm 
import torch
from torch.utils.data import random_split, DataLoader

# ===================================================
# Safety Flags for Stable GPU Training
# ===================================================
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False

# ===================================================
# Training Loop
# ===================================================
best_val_loss = float("inf")
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0

    train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch", leave=False)

    for inputs, labels in train_pbar:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Metrics
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels.data)

        train_pbar.set_postfix({
            "loss": f"{loss.item():.4f}",
            "acc": f"{(running_corrects.double() / ((len(train_pbar)*train_loader.batch_size))).item():.3f}"
        })

    epoch_loss = running_loss / len(train_ds)
    epoch_acc = running_corrects.double() / len(train_ds)

    # ===================================================
    # Validation Step
    # ===================================================
    model.eval()
    val_loss, val_corrects = 0.0, 0

    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc="Validating", unit="batch", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            val_corrects += torch.sum(preds == labels.data)

    val_loss = val_loss / len(val_ds)
    val_acc = val_corrects.double() / len(val_ds)

    print(f"Epoch {epoch+1}/{num_epochs} | "
          f"Train Loss={epoch_loss:.4f} Acc={epoch_acc:.4f} | "
          f"Val Loss={val_loss:.4f} Acc={val_acc:.4f}")

    scheduler.step(val_loss)

    # ===================================================
    # Save Best Model
    # ===================================================
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_chest_model.pth")
        print("✅ Saved best model.\n")


Epoch 1/10 | Train Loss=0.9359 Acc=0.5824 | Val Loss=0.9525 Acc=0.5417
✅ Saved best model.



Epoch 2/10 | Train Loss=0.3829 Acc=0.8793 | Val Loss=0.8011 Acc=0.6806
✅ Saved best model.



Epoch 3/10 | Train Loss=0.2282 Acc=0.9282 | Val Loss=0.4430 Acc=0.8611
✅ Saved best model.



Epoch 4/10 | Train Loss=0.1021 Acc=0.9804 | Val Loss=0.3593 Acc=0.8889
✅ Saved best model.



Epoch 5/10 | Train Loss=0.0662 Acc=0.9886 | Val Loss=0.3905 Acc=0.8889


Epoch 6/10 | Train Loss=0.0543 Acc=0.9869 | Val Loss=0.3349 Acc=0.9028
✅ Saved best model.



Epoch 7/10 | Train Loss=0.1010 Acc=0.9674 | Val Loss=0.4315 Acc=0.8611


Epoch 8/10 | Train Loss=0.0726 Acc=0.9755 | Val Loss=0.2864 Acc=0.9028
✅ Saved best model.



Epoch 9/10 | Train Loss=0.0833 Acc=0.9772 | Val Loss=0.3850 Acc=0.8750


Epoch 10/10 | Train Loss=0.0546 Acc=0.9869 | Val Loss=0.3068 Acc=0.9028


In [9]:
model.load_state_dict(torch.load("best_chest_model.pth"))
model.eval()
test_corrects = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        test_corrects += torch.sum(preds == labels.data)
        total += labels.size(0)

test_acc = test_corrects.double() / total
print("🧪 Test Accuracy:", test_acc.item())

C:\Users\vedan\AppData\Local\Temp\ipykernel_21704\3033588632.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_chest_model.pth"))


🧪 Test Accuracy: 0.8825396825396825
